## 1.0 Introduction

In this notebook we will perform all steps required to validate the finetuned GPT-3.5 Turbo model on our own custom dataset.

In [1]:
# Import Modules
import os
import numpy as np
import pandas as pd
import time
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm

# OpenAI
import openai

## 2.0 Load Dataset

We will reload the validation CSV file that was generated earlier with the notebook 'Prepare_Train_and_Validation_Datasets.ipynb'.

In [2]:
# Load Dataset
val_df = pd.read_csv('./data/val_df.csv')

# Summary
print(val_df.shape)

(1559, 11)


Let's review a small subset of the validation data...

In [3]:
val_df.head()

,id,title,text,mainSection,published_at,publisher,partisan,url,text_wordcount,max_words_text,labels
0,9266995,Verdachte dodelijke steekpartijen Maastricht l...,Verdachte dodelijke steekpartijen Maastricht l...,/nieuws,2017-12-18,ad,False,www.ad.nl/binnenland/verdachte-dodelijke-steek...,188,Verdachte dodelijke steekpartijen Maastricht l...,0
1,4130077,Honderden arrestaties bij acties tegen mensen ...,Honderden arrestaties bij acties tegen mensen ...,/nieuws,2017-02-11,ad,False,www.ad.nl/buitenland/honderden-arrestaties-bij...,122,Honderden arrestaties bij acties tegen mensen ...,0
2,11147268,Waarom de 'oudejaarsbonus' voor de jongeren va...,Waarom de 'oudejaarsbonus' voor de jongeren va...,/home,2019-01-20,trouw,True,www.trouw.nl/home/waarom-de-oudejaarsbonus-voo...,262,Waarom de 'oudejaarsbonus' voor de jongeren va...,1
3,10749100,Klaar voor de verdediging,Klaar voor de verdedigingOver ruim een week be...,/nieuws,2018-10-16,ad,False,www.ad.nl/binnenland/klaar-voor-de-verdediging...,411,Klaar voor de verdedigingOver ruim een week be...,0
4,10700707,Windvlaag grijpt springmatras en doodt 2-jarig...,Windvlaag grijpt springmatras en doodt 2-jarig...,/nieuws,2018-10-05,ad,False,www.ad.nl/buitenland/windvlaag-grijpt-springma...,286,Windvlaag grijpt springmatras en doodt 2-jarig...,0


## 3.0 Validate finetuned OpenAI GPT-3.5 Model

In this section we will validate our finetuned GPT 3.5 model against our validation dataset and determine the validation accuracy.

In [4]:
# OpenAI API Key
openai.api_key = os.environ["OPENAI_API_KEY"]

# OpenAI Model Identifier
finetuned_model_id = 'ft:gpt-3.5-turbo-0613:personal::8BTLwDu0'

In [5]:
def create_prompt(item_text, item_label = None, inference = False):
    if inference:
        # Base Prompt
        prompt_text = [{"role": "system", "content": "Je bent redacteur bij een krant. Je beoordeeld een krantenartikel of het politiek of neutraal is. Hieronder staat de tekst van het krantenartikel."}, {"role": "user", "content": ""}]

        # Set Text and Label
        prompt_text[1]['content'] = '### Tekst:\n' + item_text
    else:   
        # Base Prompt
        prompt_text = {"messages": [{"role": "system", "content": "Je bent redacteur bij een krant. Je beoordeeld een krantenartikel of het politiek of neutraal is. Hieronder staat de tekst van het krantenartikel."}, {"role": "user", "content": ""}, {"role": "assistant", "content": ""}]}

        # Set Text and Label
        prompt_text['messages'][1]['content'] = '### Tekst:\n' + item_text
        prompt_text['messages'][2]['content'] = item_label
    
    return prompt_text

### 3.1 Inference on Fine-Tuned OpenAI Model

We loop through all the records in the validation dataframe. The finetuned model will respond with either 'Politiek' or 'Neutraal' to classify the newspaper article as partisan or neutral.

For each response we determine the binary label to eventually create the Classification Report.

In [6]:
# Placeholders
gt_labels, pred_labels = [], []

# Inference on Validation Dataframe
for index, row in tqdm(val_df.iterrows(), total = val_df.shape[0]):
    text = row['max_words_text']
    label = row['labels']
    gt_labels.append(label)

    # Create Prompt
    prompt = create_prompt(text, item_label = None, inference = True)

    # Call API for inference on FineTuned Model
    # Simple error handling .... earlier this summer I experienced occasional errors with the model not responding/not being available.. A simple wait period was enough to have it working.
    # Recently no bad experiences with that...however I keep the error handling for it in-place
    try:
        completion = openai.ChatCompletion.create(model = finetuned_model_id,
                                                  messages = prompt)
    except Exception as err:
        print(err)
        time.sleep(120.0)
        completion = openai.ChatCompletion.create(model = finetuned_model_id,
                                                  messages = prompt)


    # Response Determine label based on OpenAI ChatCompletion Response
    predicted_label = -1
    prediction_text = completion.choices[0].message['content']
    if prediction_text == 'Politiek':
        predicted_label = 1
    if prediction_text == 'Neutraal':
        predicted_label = 0

    # Store Result
    pred_labels.append(predicted_label)
    #print(f'Index: {index}   GT: {label}   Pred: {predicted_label}')

    # Delay...To stay within the limits for tokens/requests per minute
    time.sleep(0.2)

  0%|          | 0/1559 [00:00<?, ?it/s]

### 3.2 Classification Results

Below our classification results. The model achieves a very nice accuracy of 90.8%.

The recall and precision scores are also very balanced.

In [7]:
# Classification Results
print(classification_report(gt_labels, pred_labels, target_names = ['Neutraal', 'Politiek'], digits = 3))

              precision    recall  f1-score   support

    Neutraal      0.917     0.894     0.905       765
    Politiek      0.900     0.922     0.911       794

    accuracy                          0.908      1559
   macro avg      0.909     0.908     0.908      1559
weighted avg      0.908     0.908     0.908      1559

